# Tutorial 2: Converting Spherical Quantities to Cartesian Quantities before Volume Rendering

This notebook provides an example of how to take Rayleigh Spherical_3D output quantities (e.g. V_r, V_theta, V_phi or B_r, B_theta, B_phi) and convert them into Cartesian quantities (e.g. V_x, V_y, V_z or B_x, B_y, B_z). This works on Spherical_3D data already interpolated onto a uniform Cartesian grid of NxNxN. This is useful particularly for looking at z quantities or using Cartesian quantities in visualization renderers to map streamlines.

Then we show a few simple examples of how to create volume renderings, modify the transfer function, and manipulate the camera angle. 


## About yt and installing yt
*This section is under development*

Developed in Python, yt is an analysis and visualization toolkit for volumetric data. To install yt on your machine and activate it, visit this website: [https://yt-project.org/doc/installing.html](https://yt-project.org/doc/installing.html) 

## Preparing your Rayleigh Spherical_3D output for volume rendering with yt
*This section is under development*

Rayleigh's Spherical_3D quantities are output in a spherical grid. Many visualization software packages do not fully support spherical grids. One solution is to interpolate the quantities onto a uniform Cartesian grid with dimensions XxYxZ. Quantity values outside the spherical volume are given values of zero and can easily be made transparent using an appropriately chosen tranfer function when rendering the visualization.

To prepare your Spherical_3D output: Interpolating Spherical_3D quantities onto a uniform Cartesian grid

1. Copy the directory **/Rayleigh/post_processing/kyle_interp** onto your local system
2. Compile the executable. We use the gcc compiler: **make -f Makefile.gfortran** 
3. Create a directory called **vis_test** (or something equally suitable) in which to run the interpolator. Copy the executable **interpolator.app** and the **input** file from **/Rayleigh/post_processing/kyle_interp** there.
4. Create a directory in **vis_test** called **Spherical_3D**. In directory **Spherical_3D** copy the 3D spherical output files you wish to interpolate, including the **_grid** file.
5. In the input file, modify the uniform radial grid resolution **nr** appropriate for your files, choose the resolution **ncube** of the resulting Cartesian cube, and modify the iteration range. When interpolating a single file, **initial_iteration** and **final_iteration** should be the same.
6. Run the interpolation code: **./interpolator.app XXXX** (Where **XXXX** is the four digit quantity code of the quantity you wish to interpolate. For example, to interpolate a radial velocity spherical 3D output of quantity code **0001**, type: **./interpolator.app 0001**) 
7. If the code executed correctly, you should have a new file in your **/vis_test/Spherical_3D** directory called **_cube**. These files are what you will import into this notebook. 
8. Copy your new **_cube** files into the directory where you will run this Jupyter Notebook.


## Reading in your prepared data and creating simple yt visualizations
*This section is under development. There are known issues with YT that need to be resolved.*

After (1) your yt installation has been activated and (2) you have interpolated your Rayleigh Spherical_3D output onto a uniform cartesian grid, you are ready to read the data into Python and begin visualizations with yt. 

First we will start by defining a class to convert Spherical_3D quantites to Cartesian quantities.

In [ ]:
# This class converts from Spherical quantities to Cartesian quantities
import numpy

class cart_field:
    def __init__(self,nx, rfile,tfile,pfile):

        self.nx = nx
        self.nz = nx
        self.ny = nx
        ny = nx
        nz = nx

        self.vx = numpy.zeros( (nx,ny,nz), dtype = 'float64' )
        self.vy = numpy.zeros( (nx,ny,nz), dtype = 'float64' )
        self.vz = numpy.zeros( (nx,ny,nz), dtype = 'float64' )

        arr = numpy.fromfile(rfile,dtype='float64')
        vr = numpy.reshape(arr,(nx,nx,nx))

        arr = numpy.fromfile(tfile,dtype='float64')
        vtheta = numpy.reshape(arr,(nx,nx,nx))

        arr = numpy.fromfile(pfile,dtype='float64')
        vphi = numpy.reshape(arr,(nx,nx,nx))

        x = numpy.linspace(-1,1,nx)
        y = numpy.linspace(-1,1,nx)
        z = numpy.linspace(-1,1,nx)

        x2d = numpy.zeros((ny,nx),dtype='float64')
        y2d = numpy.zeros((ny,nx),dtype='float64')


        # We will generate vx,vy,vz one z-slice at a time.
        # Need some 2D support arrays that are functions of x and y

        for i in range(ny):
            x2d[i,:] = x[:]
        for i in range(nx):
            y2d[:,i] = y[:]

        rcyl = numpy.sqrt(x2d**2 + y2d**2)    # cylindrical radius based on x-y
        cosphi = x2d/rcyl   # cosine of angle from x-axis in x-y plane
        sinphi = y2d/rcyl   # sine of same angle
        
        for i in range(nz):
            zval = z[i]
            r2d = numpy.sqrt(zval**2+ x2d**2+y2d**2)  # spherical radius (at this z)
            costheta = zval/r2d  # costheta
            sintheta = numpy.sqrt(1.0-costheta**2) #sintheta

            # vz = vr*costheta - vtheta*sintheta
            self.vz[i,:,:]=vr[i,:,:]*costheta - vtheta[i,:,:]*sintheta

            # v_r_cylindrical = vr*sintheta + vtheta*costheta
            vrcyl = vr[i,:,:]*sintheta+vtheta[i,:,:]*costheta

            #vx = v_r_cylindrical*costheta - vphi*sinphi
            self.vx[i,:,:] = vrcyl*cosphi -vphi[i,:,:]*sinphi

            # vy = vrcyl*sinphi + vphi*cosphi
            self.vy[i,:,:] = vrcyl*sinphi + vphi[i,:,:]*cosphi

Read in the interpolated data files and perform the Cartesian conversion.

In [ ]:
#This reads in the cubed data files
#All cube data has been interpolated onto a uniform cartesian grid with dimensions 128x128x128
rfile = '32600001_0001_cube' #V_r
tfile = '32600001_0002_cube' #V_theta
pfile = '32600001_0003_cube' #V_phi

velocity = cart_field(128,rfile,tfile,pfile) #Although labeled as 'velocity', B fields can be used here instead

Load the data into YT for volume rendering and create a YT scene.

In [ ]:
import yt
import numpy as np

cube = velocity.vz #Choose the quantity you want to volume render, here vz 
nx = 128
data = dict(velocity = (cube, "cm/s"))
bbox = numpy.array([[-1.5, 1.5], [-1.5, 1.5], [-1.5, 1.5]]) #Define a bounding box
ds = yt.load_uniform_grid(data, cube.shape, bbox=bbox, nprocs=nx) #Load a cube into YT
sc = yt.create_scene(ds, field=('velocity')) #Set up a basic Scene in YT
maxup = np.amax(cube)
minup = np.amin(cube)
print (maxup, minup) #Print min and max values in cube

Show the volume rendering and plot the default transfer function.

In [ ]:
sc.show() #Show the scene
source = sc[0]
source.tfh.plot(profile_field='velocity') #Plot the default transfer function with the field quantity histogram.

Let's create a custom transfer function and plot it. The method outlined below is closer to that of an isosurface render.

In [ ]:
#Let's try to render the scene with a new tranfer function that we define ourselves
source = sc[0]

source.set_field('velocity')
source.set_log(False) #Enforce a linear space - appears to work for the transfer function, but not for volume render

#Set the quantity bounds over which to apply the colormap 
bounds = (0., 300.)

#Set the bounds of the transfer function
tf = yt.ColorTransferFunction(bounds) 

#This is how to add new opacity points to the colormap with a gaussian shape
tf.sample_colormap(50., w=50., colormap='arbre', alpha = 0.25) #Gaussian centered at 50, width 50, and alpha of 0.25
tf.sample_colormap(100., w=50., colormap='arbre')
tf.sample_colormap(200., w=50., colormap='arbre', alpha = 1.5)
tf.sample_colormap(300., w=50., colormap='arbre')

source.tfh.tf = tf
source.tfh.bounds = bounds

source.tfh.plot(profile_field='velocity') #Plot the new transfer function

Now render the scene with the new transfer function applied.

In [ ]:
# Grab the first render source and set it to use the new transfer function
render_source = sc.get_source()
render_source.transfer_function = tf

sc.render()
sc.show(sigma_clip=2.0) #Attempt a better contrast with sigma clipping

Render the same scene but looking down from the North pole. To accomplish this, we must move the camera.

In [ ]:
# Try to change view to see something more interesting
# pitch the camera by pi / 4 radians:
sc.camera.pitch(np.pi/4.0) #observe from pole - move camera. 
#Note that every time this is run, the camera will move again by pi/4.

sc.render()
sc.show(sigma_clip=2.0)